In [1]:
#pip install torch
#pip install bert-extractive-summarizer
!pip install transformers
!pip install datasets
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 1.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/vgupta123/sumpubmed.git

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 12.07 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.


In [3]:
import os
from datasets import Dataset
import pandas as pd
import re
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [4]:
def load_text_files(directory):
    files = os.listdir(directory)
    files = [file for file in files if not file.startswith('.DS')]
    files = sorted(files, key=lambda x: int(re.split('\.|_', x)[1]))
    print(len(files))
    data = []
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(directory, file), 'r', encoding='utf-8',errors='ignore') as f:
                text = f.read()
                data.append(text)
    return data


In [5]:
#print(re.split('\.|_', 'text_1010.txt'))

In [22]:
# Load text files from directories
original_text = load_text_files('/content/sumpubmed/abstract')
summaries = load_text_files('/content/sumpubmed/shorter_abstract')
# original_text = load_text_files('/content/sumpubmed/text')
# summaries = load_text_files('/content/sumpubmed/abstract')
print(len(original_text))
print(len(summaries))

# Create a dataset from the loaded text files
data = {'original_text': original_text, 'summary': summaries}

32689
32689
32689
32689


In [23]:
# Create a dataset from the loaded text files
df =pd.DataFrame({'original_text': original_text, 'summary': summaries})

# dataset = Dataset.from_dict(data)

print(df.shape)



(32689, 2)


In [24]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # or specify another GPT model suitable for text generation
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [9]:
# Tokenize the dataset
# def tokenize_function(ds):
#     return tokenizer(ds["original_text"] + " [SEP] " + ds["summary"])

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Save the tokenized dataset to a text file
# tokenized_datasets.save_to_disk("/content/")

In [25]:
df['input_text'] = df["original_text"].iloc[:1000] + " TL;DR " + df["summary"].iloc[:1000]
df["input_text"].to_csv("/content/combined_text.txt", index=False, header=False)

In [11]:
# df['input_text'].iloc[:2]
df.shape

(32689, 3)

In [26]:
# Load your dataset
train_file = "/content/combined_text.txt"
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [27]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [28]:
# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="/content/gpt2-finetuned-summarization",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10000,
    save_total_limit=2,
)

In [29]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [30]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("/content/gpt2-finetuned-summarization")
tokenizer.save_pretrained("/content/gpt2-finetuned-summarization")

Step,Training Loss
500,3.638300
1000,3.470100
1500,3.401900
2000,3.359100
2500,3.313900
3000,3.300700
3500,3.261500
4000,3.254200
4500,3.239100
5000,3.216700


('/content/gpt2-finetuned-summarization/tokenizer_config.json',
 '/content/gpt2-finetuned-summarization/special_tokens_map.json',
 '/content/gpt2-finetuned-summarization/vocab.json',
 '/content/gpt2-finetuned-summarization/merges.txt',
 '/content/gpt2-finetuned-summarization/added_tokens.json')

In [31]:
# Load the fine-tuned model and tokenizer
model_path = "/content/gpt2-finetuned-summarization"  # Replace with the actual path to your fine-tuned model
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [32]:
input_text = df['original_text'][10000]
print(df['original_text'][10000])

BACKGROUND
in mammals, pheromones play an important role in social and innate reproductive behavior within species. in rodents, vomeronasal receptor type  <dig> , which is specifically expressed in the vomeronasal organ, is thought to detect pheromones. the v1r gene repertoire differs dramatically between mammalian species, and the presence of species-specific v1r subfamilies in mouse and rat suggests that v1r plays a profound role in species-specific recognition of pheromones. in ruminants, however, the molecular mechanism for pheromone perception is not well understood. interestingly, goat male pheromone, which can induce out-of-season ovulation in anestrous females, causes the same pheromone response in sheep, and vice versa, suggesting that there may be mechanisms for detecting "inter-species" pheromones among ruminant species.


RESULTS
we isolated  <dig> goat and  <dig> sheep intact v1r genes based on sequence similarity with  <dig> cow v1r genes in the cow genome database. we fo

In [33]:
# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=False)

# Generate the summary
summary_ids = model.generate(input_ids, max_new_tokens=512 , num_beams=4, length_penalty=2.0, early_stopping=True) #max_length=512

# Decode and print the generated summary
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(generated_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BACKGROUND
in mammals, pheromones play an important role in social and innate reproductive behavior within species. in rodents, vomeronasal receptor type  <dig>, which is specifically expressed in the vomeronasal organ, is thought to detect pheromones. the v1r gene repertoire differs dramatically between mammalian species, and the presence of species-specific v1r subfamilies in mouse and rat suggests that v1r plays a profound role in species-specific recognition of pheromones. in ruminants, however, the molecular mechanism for pheromone perception is not well understood. interestingly, goat male pheromone, which can induce out-of-season ovulation in anestrous females, causes the same pheromone response in sheep, and vice versa, suggesting that there may be mechanisms for detecting "inter-species" pheromones among ruminant species.


RESULTS
we isolated  <dig> goat and  <dig> sheep intact v1r genes based on sequence similarity with  <dig> cow v1r genes in the cow genome database. we fou